In [64]:
from pathlib import Path
from utils import (
    loads_cae_db,
    executes_query_duckdb
)

In [65]:
cae_path = Path(
    "../../kids_chances/data/raw/cae_history.txt"
)

destionation_db = Path("../data/cae_db.duckdb")
cae = loads_cae_db(cae_path, destionation_db)

In [66]:
query = """
select distinct(tipo_beneficiario) from cae"""
executes_query_duckdb(query, cae)

,tipo_beneficiario
0,NUEVO BENEFICIARIO
1,EGRESO
2,SIN CREDITO POR NO MATRICULA O POR SUSPENSION
3,BENEFICIARIO RENOVANTE
4,DESERCION
5,GRATUIDAD


In [86]:
query_new = """
WITH summary_table as (
    SELECT
        id_2023 AS id,
        genero_registrado AS genero, 
        region_grupo_familiar AS region,
        quintil_ingreso_grupo_familiar AS quintil,
        tipo_ies AS tipo_establecimiento,
        region_sede_ies AS region_estudio,
        tipo_titulo AS tipo_carrera,
        dependencia_colegio AS tipo_colegio,
        nombre_carrera as carrera,
        nombre_ies as establecimiento,

        SUM(arancel_solicitado) as total_borrowed,
        
        MIN(CASE WHEN tipo_beneficiario = 'NUEVO BENEFICIARIO' THEN año_licitacion END)
            AS año_inicio_cae,

        MIN(CASE WHEN tipo_beneficiario = 'DESERCION' THEN año_operacion END)
            AS año_desercion,

        COUNT(CASE WHEN tipo_beneficiario IN ('BENEFICIARIO RENOVANTE','NUEVO BENEFICIARIO')
                THEN tipo_beneficiario END)
            AS años_financiados,

        MIN(CASE WHEN tipo_beneficiario = 'EGRESO' THEN año_operacion END)
            AS año_egreso,

        MIN(CASE WHEN tipo_beneficiario = 'GRATUIDAD' THEN año_operacion END)
            AS año_gratuidad,

        CASE 
            WHEN SUM(CASE WHEN cambio_ies_carrera = 'SI' THEN 1 ELSE 0 END) > 0
                THEN 'SI'
            ELSE 'NO'
        END AS cambio_ies_carrera,

    FROM cae
    GROUP BY
        id, genero, region, quintil, tipo_establecimiento, region_estudio,
        tipo_carrera, tipo_colegio, carrera, establecimiento),
    
    total_años_financiados as (
        SELECT id, SUM(años_financiados) as total_años_financiados
        FROM summary_table
        GROUP BY id
        )
    
        SELECT st.*,
            taf.total_años_financiados
    FROM summary_table st
    LEFT JOIN total_años_financiados taf
        ON st.id = taf.id
        """

summary_table = executes_query_duckdb(query_new, cae)

In [84]:
summary_table

,id,genero,region,quintil,tipo_establecimiento,region_estudio,tipo_carrera,tipo_colegio,carrera,establecimiento,total_borrowed,año_inicio_cae,año_desercion,años_financiados,año_egreso,año_gratuidad,cambio_ies_carrera
0,672.837,F,REGION DE LA ARAUCANIA,quintil 1 y 2,U. PRIVADA - NO CRUCH,9,Profesional,Municipal,INGENIERIA EN ADMINISTRACION DE EMPRESAS MENCI...,UNIVERSIDAD TECNOLÓGICA DE CHILE INACAP,606000.0,2012,2015,2,<NA>,<NA>,NO
1,688.877,F,REGION DEL BIO-BIO,quintil 1 y 2,IP,8,Técnico,Municipal,TECNICO EN ODONTOLOGIA,INSTITUTO PROFESIONAL AIEP,820000.0,2013,<NA>,2,2015,<NA>,NO
2,575.448,M,REGION DE LOS LAGOS,quintil 1 y 2,U. PRIVADA - NO CRUCH,10,Profesional con Licenciatura,Municipal,DERECHO,UNIVERSIDAD SAN SEBASTIÁN,11664014.0,2012,<NA>,5,2017,<NA>,NO
3,610.556,F,REGION DE COQUIMBO,quintil 1 y 2,CFT,4,Técnico,Particular subvencionado,TECNICO EN ENFERMERIA,CENTRO DE FORMACIÓN TÉCNICA SANTO TOMÁS,1671000.0,2012,<NA>,2,2014,<NA>,NO
4,624.335,F,REGION METROPOLITANA,quintil 4,U. PRIVADA - NO CRUCH,13,Profesional con Licenciatura,Particular pagado,LICENCIATURA EN TEORIA E HISTORIA DEL ARTE,UNIVERSIDAD ALBERTO HURTADO,2159871.0,<NA>,<NA>,1,<NA>,2016,SI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681570,691.730,F,REGION METROPOLITANA,quintil 1 y 2,IP,13,Profesional,Particular subvencionado,AUDITORIA,INSTITUTO PROFESIONAL DUOC UC,0.0,<NA>,<NA>,0,<NA>,2018,NO
1681571,420.832,F,REGION DE VALPARAISO,quintil 4,IP,5,Profesional,Municipal,INGENIERIA EN COMERCIO EXTERIOR,INSTITUTO PROFESIONAL DUOC UC,1058270.0,2019,<NA>,1,<NA>,<NA>,NO
1681572,302.659,F,REGION DE LOS RIOS,quintil 4,U. PRIVADA - NO CRUCH,14,Profesional con Licenciatura,Municipal,PSICOLOGIA,UNIVERSIDAD SANTO TOMÁS,2498514.0,2019,<NA>,1,<NA>,<NA>,NO
1681573,1.042.882,F,REGION METROPOLITANA,quintil 5,U. PRIVADA - NO CRUCH,13,Bachiller,Particular pagado,BACHILLERATO EN PSICOLOGIA,UNIVERSIDAD DEL DESARROLLO,2447185.0,2019,<NA>,1,<NA>,<NA>,NO
